# Custom Classes: Transformers

We'll be introducing some new tools to implement what we did last session. Using these custom classes (regressors, classifiers, cluster-ers, transformers, feature unions and pipelines) can be powerful additions to your tool belt.

This introduction is modeled after Adam Rogers's titanic_finished-ish.py script we worked through last time.

We start by pulling in the datasets and importing our libraries. Data available at https://www.kaggle.com/c/titanic/data.

In [364]:
import sklearn as sk
import pandas as pd
import numpy as np

In [365]:
train = pd.read_csv('../titanic/data/train.csv')
test = pd.read_csv('../titanic/data/test.csv')

# combining early to apply transformations uniformly
combinedSet = pd.concat([train , test], axis=0)
combinedSet = combinedSet.reset_index(drop=True)

As a reminder this set includes:

| Variable      | Description  |  Values  |
| ------------- |:-------------:| -----:|
| survived      | Survival | (0 = No; 1 = Yes) |
| pclass     | Passenger Class     |   (1 = 1st; 2 = 2nd; 3 = 3rd) |
| name  | Name     |    String |
| sex | Sex      |    ('male' or 'female') |
| age | Age     |    Float 0-80  |
| sibsp | Number of Siblings/Spouses Aboard      |    Int |
| parch | Number of Parents/Children Aboard      |    Int |
| ticket | Ticket Number      |    String  |
| fare | Passenger Fare      |    Float |
| cabin| Cabin     |    String (e.g. C134) |
| embarked| Port of Embarkation      |    ('C' = Cherbourg; 'Q' = Queenstown; 'S' = Southampton) |


In [366]:
combinedSet.shape

(1309, 12)

In [367]:
combinedSet['Survived'].value_counts(dropna=False)

 0.0    549
NaN     418
 1.0    342
Name: Survived, dtype: int64

In [368]:
combinedSet[450:453]

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
450,36.0,NaN,S,27.7500,"West, Mr. Edwy Arthur",2,451,2,male,1,0.0,C.A. 34651
451,NaN,NaN,S,19.9667,"Hagland, Mr. Ingvald Olai Olsen",0,452,3,male,1,0.0,65303
452,30.0,C111,C,27.7500,"Foreman, Mr. Benjamin Laventall",0,453,1,male,0,0.0,113051


In [369]:
combinedSet.describe()

,Age,Fare,Parch,PassengerId,Pclass,SibSp,Survived
count,1046.000000,1308.000000,1309.000000,1309.000000,1309.000000,1309.000000,891.000000
mean,29.881138,33.295479,0.385027,655.000000,2.294882,0.498854,0.383838
std,14.413493,51.758668,0.865560,378.020061,0.837836,1.041658,0.486592
min,0.170000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
25%,21.000000,7.895800,0.000000,328.000000,2.000000,0.000000,0.000000
50%,28.000000,14.454200,0.000000,655.000000,3.000000,0.000000,0.000000
75%,39.000000,31.275000,0.000000,982.000000,3.000000,1.000000,1.000000
max,80.000000,512.329200,9.000000,1309.000000,3.000000,8.000000,1.000000


## Transformers
Transformers apply a transformation on the data. We did several 'transformations' on the data last session to prepare it for model fitting. These transformation included:

* filling missing age values;
* converting Pclass, Embarked, and Deck to indicator variables with pd.get_dummies() (S -> [0, 0, 1])
* converting gender to a binary variable
* creating an IsChild indicator variable



Many other transformers are included in sklearn.preprocessing package (```StandardScaler()```, ```Binarizer()```, ```OneHotEncoder()```, ```Imputer()```, ```LabelEncoder()```, etc.). The feature extraction sklearn module also has many invaluble transformers. Several of which that are great for natural language processing (```TfidfTransformer()```, ```CountVectorizer()```, ```HashingVectorizer()```, etc.).

These transformers within Sci-kit Learn are classes that have useful methods that can make preparing your data a bit easier. Many tranformers have the ```fit()```, ```transform()```, ```fit_transform()```, and ```inverse_transform()``` methods.

Last week we had an issue when the test split did not have the same levels of cabin as the train set. One benefit of the transformer model is that it can fit on train data and the same model will be used to map any new data.

If you've ever had to scale your labels and want to get the predicted values back out in the previous scale inverse_transform() helps immensely.

Transformation Benefits:

- better with training with one set and transforming new data
- can allow for inversing the transformation
- can allow for more abstraction for common transformations
- works with pipelines (to be discussed later)

### Some examples:

Simple scaling:

In [370]:
from sklearn import preprocessing
pd.DataFrame({'Fare': train.Fare,\
             'ScaledFare': preprocessing.scale(train.Fare)})\
                .head()

,Fare,ScaledFare
0,7.2500,-0.502445
1,71.2833,0.786845
2,7.9250,-0.488854
3,53.1000,0.420730
4,8.0500,-0.486337


But what if we need to scale new data or  we were predicting fare and need to retrun to the previous scale.

New data set scaling and reversible scaling:

In [371]:
# instantiate
scaler = preprocessing.StandardScaler()

#fit
scaler.fit(train.Fare.reshape(-1, 1)) # reshape(-1, 1) to get column vector for scaling
testFares =  test.Fare.fillna(0).reshape(-1, 1) # had to fill NA as no NA's in train set, could impute instead

#transform
scaledFare = scaler.transform(testFares)

#inverse transform
inverseScaled = scaler.inverse_transform(scaledFare)

#different from scaling separately, shown if we fit_transform with test alone
scaler2 = preprocessing.StandardScaler()
badScaling = scaler2.fit_transform(testFares)

pd.DataFrame({'Fare': test.Fare.values,\
             'ScaledFare': scaledFare.T[0],\
             'InverseFare': inverseScaled.T[0],
             'BadScaling': badScaling.T[0]},\
             columns=['Fare', 'ScaledFare','InverseFare', 'BadScaling'])\
                .head()


,Fare,ScaledFare,InverseFare,BadScaling
0,7.8292,-0.490783,7.8292,-0.496637
1,7.0000,-0.507479,7.0000,-0.511497
2,9.6875,-0.453367,9.6875,-0.463335
3,8.6625,-0.474005,8.6625,-0.481704
4,12.2875,-0.401017,12.2875,-0.416740


Still can be prone to issues if new data has levels/non-numbers not present in train data eg. NaN's for the example above.

So it is apparent they can add some functionality what about custom transformers?

### Custom Transformers

Can be done simply using ```sklearn.preprocessing.FunctionTransformer()``` or more customization is available from creating a new class inheriting from the base classes: BaseEstimator + what type of estimator you are creating (ClassifierMixin, ClusterMixin, RegressorMixin, TransformerMixin)  http://scikit-learn.org/stable/modules/classes.html

Function transformer example:

In [372]:
genderDict = {'male': 1, 'female': 0}
genderFlagger = sk.preprocessing.FunctionTransformer(lambda genderArray: \
                                                   [genderDict[gender] for gender in genderArray],\
                                                   validate=False)

In [373]:
genderFlags= genderFlagger.transform(combinedSet.Sex)
print genderFlags[0:10]

[1, 0, 0, 0, 1, 1, 1, 1, 0, 0]


Not simpler than 
```python
passengers["Sex"][passengers["Sex"] == 'male'] = 0
passengers["Sex"][passengers["Sex"] == 'female'] = 1
```
Pandas makes FunctionTransformer sort of obsolete.

For more customization we create a new class:
``` python                                                                                                                                        
class NewTransformer(base.BaseEstimator, base.ClassifierMixin):
    def __init__(self, ...):
        # initialization code

    def fit(self, X, y=None):
        # fit the model ...
        return self

    def transform(self, X):
        # transformation
        return new_X
    
    def fit_transform(self, X, y=None):
        # fit the model and then transform it
        return new_X 
```

Little more complex for cabin deck.

In [374]:
combinedSet['Cabin'].value_counts(dropna=False)[0:10]

NaN                1014
C23 C25 C27           6
B57 B59 B63 B66       5
G6                    5
C78                   4
B96 B98               4
C22 C26               4
F33                   4
D                     4
F4                    4
Name: Cabin, dtype: int64

In [375]:
class cabinLevelsTransformer1(sk.base.BaseEstimator, sk.base.ClassifierMixin):
    
    # function for extracting deck letter
    def get_deck_letter(self, row):
        # Ignore NaN values
        if not pd.isnull(row["Cabin"]):
            # Get first letter of "Cabin" value
            return str(row["Cabin"])[0]
        # Otherwise return NaN
        return row["Cabin"]        

    def transform(self, X):
        # transformation
        newX=X.copy()
        newX["Cabin"] = newX.apply(lambda row: self.get_deck_letter(row), axis=1)
        cabinColumnsDF = pd.get_dummies(newX, columns = ["Cabin"], prefix=['cabin'])                             
        return cabinColumnsDF

In [376]:
class cabinLevelsTransformer2(sk.base.BaseEstimator, sk.base.ClassifierMixin):
    def __init__(self):
        # initialization code
        self.le = preprocessing.LabelEncoder()
        self.lb = preprocessing.LabelBinarizer()
        
    # function for extracting deck letter
    def get_deck_letter(self, row):
        # Ignore NaN values
        if not pd.isnull(row["Cabin"]):
            # Get first letter of "Cabin" value
            return str(row["Cabin"])[0]
        # Otherwise return NaN
        return 'NaN'        

    def fit(self, X, y = None):
        # fit the model ...
        newX=X.copy()
        newX["Cabin"] = newX.apply(lambda row: self.get_deck_letter(row), axis=1)
        self.le.fit(newX["Cabin"])
        self.lb.fit(self.le.transform(newX["Cabin"]))
        return self

    def transform(self, X):
        # transformation
        newX=X.copy()
        newX["Cabin"] = newX.apply(lambda row: self.get_deck_letter(row), axis=1)
        wtf=self.le.transform(newX["Cabin"])
        cabinColumns = self.lb.transform(wtf)
        cabinColumnsDF = pd.DataFrame(cabinColumns)
        cabinColumnsDF.columns = ['cabin_' + str(cabinNum) for cabinNum in self.le.inverse_transform(cabinColumnsDF.columns)]
        newX = newX.drop('Cabin',1)
        newX = pd.concat([newX.reset_index(drop=True), cabinColumnsDF], axis=1) 
        return newX.drop('cabin_NaN', 1)

    def fit_transform(self, X, y=None):
        # fit the model and then transform it
        self.fit(X)
        return  self.transform(X)


In [377]:
clTrans1=cabinLevelsTransformer1()
clTrans2=cabinLevelsTransformer2()

In [378]:
clTrans1.transform(combinedSet).head(2)

,Age,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,cabin_A,cabin_B,cabin_C,cabin_D,cabin_E,cabin_F,cabin_G,cabin_T
0,22.0,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,38.0,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


The more complex cabinLevelsTransformer2 allows for transforming on a set that may not include all the cabin levels. Note: Label Encoder seems to not play well with transforming NaN's so NaN's should be labelled say 'NaN' in the transformer.

In [379]:
clTrans2.fit(train)

cabinLevelsTransformer2()

In [380]:
test.loc[12:14,:]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
12,904,1,"Snyder, Mrs. John Pillsbury (Nelle Stevenson)",female,23.0,1,0,21228,82.2667,B45,S
13,905,2,"Howard, Mr. Benjamin",male,63.0,1,0,24065,26.0000,NaN,S
14,906,1,"Chaffee, Mrs. Herbert Fuller (Carrie Constance...",female,47.0,1,0,W.E.P. 5734,61.1750,E31,S


In [381]:
clTrans2.transform(test.loc[12:14,:])

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,cabin_A,cabin_B,cabin_C,cabin_D,cabin_E,cabin_F,cabin_G,cabin_T
0,904,1,"Snyder, Mrs. John Pillsbury (Nelle Stevenson)",female,23.0,1,0,21228,82.2667,S,0,1,0,0,0,0,0,0
1,905,2,"Howard, Mr. Benjamin",male,63.0,1,0,24065,26.0000,S,0,0,0,0,0,0,0,0
2,906,1,"Chaffee, Mrs. Herbert Fuller (Carrie Constance...",female,47.0,1,0,W.E.P. 5734,61.1750,S,0,0,0,0,1,0,0,0


Keep in mind we don't want to drop any additional columns as NaN's are represented as 0's across cabins. 

Our transformer looks like it'd be pretty similar for any categorical columns. We can make a more generic version. 

In [382]:
class genericLevelsToDummiesTransformer(sk.base.BaseEstimator, sk.base.ClassifierMixin):
    def __init__(self, columns, printFlag = False):
        # initialization code
        self.columns = columns
        self.leDict = {}
        self.lbDict = {}
        self.printFlag = printFlag
        self.newColumnNames = {}
        for column in columns:
            # unique transformers for each column
            self.leDict[column] = preprocessing.LabelEncoder()
            self.lbDict[column] = preprocessing.LabelBinarizer()
        
    # function for extracting deck letter
    def get_deck_letter(self, row):
        # Ignore NaN values
        if not pd.isnull(row["Cabin"]):
            # Get first letter of "Cabin" value
            return str(row["Cabin"])[0]
        # Otherwise return NaN
        return 'NaN'    
    
    def fit(self, X, y = None):
        # fit the model ...
        newX=X.copy()
        for column in self.columns:
            if column == 'Cabin':
                newX["Cabin"] = newX.apply(lambda row: self.get_deck_letter(row), axis=1)
            self.leDict[column].fit(newX[column])
            self.lbDict[column].fit(self.leDict[column].transform(newX[column]))
        return self

    def transform(self, X):
        # transformation
        newX=X.copy()
        if self.printFlag: print newX
        for column in self.columns:
            
            if column == 'Cabin':
                newX["Cabin"] = newX.apply(lambda row: self.get_deck_letter(row), axis=1)
            
            # convert to numeric    
            newX[column] = self.leDict[column].transform(newX[column])
            if self.printFlag: print newX[column]
                
            # make dummies
            newColumnsDF = pd.DataFrame(self.lbDict[column].transform(newX[column]))
            
            # rename dummies to original category levels
            self.newColumnNames[column]=[column+ '_' + str(index) \
                                         for index in self.leDict[column].inverse_transform(newColumnsDF.columns)]
            newColumnsDF.columns = self.newColumnNames[column]
            if self.printFlag: print newColumnsDF
            
            newX = newX.drop(column,1)
            newX = pd.concat([newX.reset_index(drop=True), newColumnsDF], axis=1) 
        return newX

    def fit_transform(self, X, y=None):
        # fit the model and then transform it
        self.fit(X)
        return  self.transform(X)
    
    def inverse_transform(self, X):
        newX=X.copy()
        for column in self.columns:
            if self.printFlag: print newX.loc[:,self.newColumnNames[column]].values
            invNumColumn = self.lbDict[column].inverse_transform(newX.loc[:,self.newColumnNames[column]].values)
            if self.printFlag: print invNumColumn
            invColumnDF = pd.Series(self.leDict[column].inverse_transform(invNumColumn), name=column)
            if self.printFlag: print invColumnDF
            for dropColumn in self.newColumnNames[column]:
                if dropColumn in newX.columns:
                    newX = newX.drop(dropColumn,1)
            newX = pd.concat([newX.reset_index(drop=True), invColumnDF], axis=1)            
        return newX
                

In [383]:
dummyTransformer=genericLevelsToDummiesTransformer(['Cabin','Sex', 'Pclass','Embarked'], printFlag=False)

In [384]:
dummyTransformer.fit(combinedSet)

genericLevelsToDummiesTransformer(columns=['Cabin', 'Sex', 'Pclass', 'Embarked'],
                 printFlag=False)

In [385]:
dummyTransformer.transform(test).head(3)

,PassengerId,Name,Age,SibSp,Parch,Ticket,Fare,Cabin_A,Cabin_B,Cabin_C,...,Cabin_NaN,Cabin_T,Sex_female,Pclass_1,Pclass_2,Pclass_3,Embarked_nan,Embarked_C,Embarked_Q,Embarked_S
0,892,"Kelly, Mr. James",34.5,0,0,330911,7.8292,0,0,0,...,1,0,1,0,0,1,0,0,1,0
1,893,"Wilkes, Mrs. James (Ellen Needs)",47.0,1,0,363272,7.0000,0,0,0,...,1,0,0,0,0,1,0,0,0,1
2,894,"Myles, Mr. Thomas Francis",62.0,0,0,240276,9.6875,0,0,0,...,1,0,1,0,1,0,0,0,1,0


In [386]:
dummyTransformer.inverse_transform(dummyTransformer.transform(train).head(3))

,PassengerId,Survived,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Sex,Pclass,Embarked
0,1,0,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,male,3,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C,female,1,C
2,3,1,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,female,3,S


In [387]:
combinedSet.head(3)

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282


The nice thing is now if we realize a new categorical feature we'd like to add it's a two line command after creating the categorical column.

Like titles in the name...

In [388]:
import operator
words = [word for this_name in combinedSet.Name for word in this_name.split(' ')]
d = {}
for word in words:
    if word in d.keys():
        d[word] += 1
    else:
        d[word] = 1

sorted_words = sorted(d.items(), key = operator.itemgetter(1), reverse = True)
sorted_words[1:10]

[('Miss.', 260),
 ('Mrs.', 197),
 ('William', 85),
 ('John', 72),
 ('Master.', 61),
 ('Henry', 47),
 ('Charles', 38),
 ('James', 37),
 ('George', 35)]

In [393]:
import re
def get_title(row):
    if not pd.isnull(row["Name"]):
        reResult = re.findall(r'Mr\.|Mrs\.|Rev\.|Miss\.|Jr|Dr\.|Rev.|Master', row["Name"])
        if len(reResult)<1:
            return 'NaN'
        else:
            return reResult[0]
combinedSet['Title'] = combinedSet.apply(lambda row: get_title(row), axis=1)

In [394]:
dummyTransformer=genericLevelstoNumTransformer(['Cabin','Sex', 'Pclass','Embarked', 'Title'], printFlag=False)

In [395]:
dummyTransformer.fit_transform(combinedSet).head(3)

,Age,Fare,Name,Parch,PassengerId,SibSp,Survived,Ticket,Cabin_A,Cabin_B,...,Embarked_C,Embarked_Q,Embarked_S,Title_Dr.,Title_Master,Title_Miss.,Title_Mr.,Title_Mrs.,Title_NaN,Title_Rev.
0,22.0,7.2500,"Braund, Mr. Owen Harris",0,1,1,0.0,A/5 21171,0,0,...,0,0,1,0,0,0,1,0,0,0
1,38.0,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,1.0,PC 17599,0,0,...,1,0,0,0,0,0,0,1,0,0
2,26.0,7.9250,"Heikkinen, Miss. Laina",0,3,0,1.0,STON/O2. 3101282,0,0,...,0,0,1,0,0,1,0,0,0,0


In [396]:
dummyTransformer.fit_transform(combinedSet).columns

Index([u'Age', u'Fare', u'Name', u'Parch', u'PassengerId', u'SibSp',
       u'Survived', u'Ticket', u'Cabin_A', u'Cabin_B', u'Cabin_C', u'Cabin_D',
       u'Cabin_E', u'Cabin_F', u'Cabin_G', u'Cabin_NaN', u'Cabin_T',
       u'Sex_female', u'Pclass_1', u'Pclass_2', u'Pclass_3', u'Embarked_nan',
       u'Embarked_C', u'Embarked_Q', u'Embarked_S', u'Title_Dr.',
       u'Title_Master', u'Title_Miss.', u'Title_Mr.', u'Title_Mrs.',
       u'Title_NaN', u'Title_Rev.'],
      dtype='object')

## On to [Estimators](https://github.com/SethPaul/scikitFlowDemo/blob/master/estimators.ipynb)